# Building A Port Scanner:
----
Reconnaisance severs as the first step 
Attackers must discover where the vulberabilities are in a system as well as systeom admins needs to know how to harden their systems down

We will build a small recon script that scans a target host for an open TCP port.

To do that we need to build sockets. 

Python provides access to BSD socet interfaces. THese provide an application-programming interface to allow coders to write applicatinost in order to perform network communications between hosts. 

A series of socket API functions can create functions to bind, linsten, connect, or send traffic on TCP/IP sockets.

The majority of the internet is accessible applciatinos on TCP. 

In a taret organization, the web server might reside on TCP port 80, the email server on TCP port 25, and file transfer on 21. 

An attacker must know both the Internet Protocol Address and the TCP port assosciated with the service. 

Attackers routinely perform a port scan in the opening stages of an attack. One type of port scan involves sending TCP SYN packets to a series of common ports and waiting for the ACK response to signal an iopen port. 

TCP Connect Scan uses the full three way handshake to determine the availablility of the service or port. 

## TCP FUll Connect Scan
----
Start by writing a TCP Port Scanner that will utilize a TCP full connect scan to identify hosts. Python has as a BSD socket API that will provide the functions necessary to make the connection.
you can view this with the command
`help('socket')`

Specificially: `socket.gethostbyname(hostname)`, `socket.gethostbyaddr(ip addr)`, `socket.create_connection()`

To better understand how the Port Scanner works, break the script up into 5 main components or steps

1. Input a hostname and a commaa separated list of ports to scan
2. Translate each name into the IPV4 Address
3. Connect to each target address and port
4. determine the specific service by sending data and reading the banner
5. print the response

`optparse` offers us the ability to script in command line tools options we want our user to run. 

`help('optparse')`

In [3]:
import optparse
    parser = optparse.OptionParser('usage %prog '+\
                                  '-H <target host> -p <target port>')
    parser.add_option('-H', dest='tgtHost', type='string', help='specify target host')
    parser.add_option('-p', dest='tgtPort', type='string', help='specifiy target port[s] separated by comma')
    
    (options, args) = parser.parse_args()
    tgtHost = options.tgtHost
    tgtPort = option.tgtPort
    
    if (tgtHost == None) | (tgtPort == None):
        print parser.usgage
        exit(0)

IndentationError: unexpected indent (<ipython-input-3-a4213aa318be>, line 2)

now that we have the command line options, define two functions to perform the tasks we need
`connScan` and `portScan`

**portScan()**: WIll take in an IP address and try to resolve it's friendly name using the gethostbyname(), then it will print the hostname or the IP address and then enumerate through each individual port attempting to connect using the `connScan()`.

**connScan()**: takes in two arguments `tgtHost` and `tgtPort`. It will attempt to create a coonection using these arguments


----


In [4]:
import optparse
from socket import *

def connScan(tgtHost, tgtPort):
    try:
        connSkt = socket(AF_INET, SOCK_STREAM)
        connSkt.connect((tgtHost, tgtPort))
        print('[+] %d/ tcp open'% tgtPort)
        connskt.close()
    except:
        print( '[-] %d/tcp closed'% tgtPort)
def portScan(tgtHost, tgtPorts):
    try:
        tgtIP = gethostbyname(tgtHost)
    except:
        print("[-] Cannot resolve '%s': Unkown host"%tgtHost)
        return
    try:
        tgtName = gethostbyaddr(tgtIP)
        print('\n[+] Scan Results for: ' + tgtName[0])
    except: 
        print('\n[+] Scan Results for: ' + tgtIP)
        setdefaulttimeout(1)
        for tgtPort in tgtPorts:
            print('Scanning port ' + tgtPort)
            connScan(tgtHost, int(tgtPort))

## Application Banner Grabbing
----
In order to grab the application banner from out target host, we must first insert additional code into the `connScan()`

After we find an open port we send a string of data to the port to see what the response is. This is to help us narrow down the range of things that might be running on it. 

In [8]:
# Try running this is in the command line with flags for IP and ports

import optparse
import socket 
from socket import *

def connScan(tgtHost, tgtPort):
    try:
        connSkt = socket(AF_INET, SOCKET_STREAM)
        connSkt.connect((tgtHost, tgtPort))
        connSkt.send('ViolentPython\r\n')
        results = connSkt.recv(100)
        print('[+] %d/tcp open'% tgtPort)
        print('[+] ' + str(results))
        connSkt.close()
    except:
        print('[-]%d/tcp close'% tgtPort)
def portScan(tgtHost, tgtPorts):
    try:
        tgtIP = gethostbyname(tgtHost)
    except:
        print("[-] Cannot resolve '%s' : Unknown host" %tgtHost)
        return
    try:
        tgtName = gethostbyaddr(tgtIP)
        print('\n[+] Scan Results for: ' + tgtName[0])
    except:
        print('\n[+] Scan Results for: ' + tgtIP)
    setdefaulttimeout(1)
    for tgtPort in tgtPorts:
        print('Scanning Port ' + tgtPort)
        connScan(tgtHost, int(tgtPort))
def main():
    parser = optparse.OptionParser("usage%prog " +\
                                  "-H <targethost> -p <target port>")
    parser.add_option('-H', dest='tgtHost', type='string', help='specify target host')
    parser.add_option('-p', dest='tgtPort', type='string', help='specifiy target port[s] separated by comma')
    
    (options, args) = parser.parse_args()
    tgtHost = options.tgtHost
    tgtPorts = str(options.tgtPort).split(', ')
    if (tgtHost == None ) | (tgtPorts[0] == None):
        print('[-] Youe must specify a target host and port[s].')
        exit(0)
    portScan(tgtHost, tgtPorts)
    if __name__ == '__main__':
        main()

## Threading the Scan to allow for multiple execution
----
Depending on the timeout variable for a sokcket a scan of each socket can take a bit. 
This can quickly add up

Threading provdes a way to utilize simultaneous execution. To use this we add in the iteration loop of portScan()


In [ ]:
import optparse
import socket 
from socket import *

def connScan(tgtHost, tgtPort):
    try:
        connSkt = socket(AF_INET, SOCKET_STREAM)
        connSkt.connect((tgtHost, tgtPort))
        connSkt.send('ViolentPython\r\n')
        results = connSkt.recv(100)
        print('[+] %d/tcp open'% tgtPort)
        print('[+] ' + str(results))
        connSkt.close()
    except:
        print('[-]%d/tcp close'% tgtPort)
def portScan(tgtHost, tgtPorts):
    try:
        tgtIP = gethostbyname(tgtHost)
    except:
        print("[-] Cannot resolve '%s' : Unknown host" %tgtHost)
        return
    try:
        tgtName = gethostbyaddr(tgtIP)
        print('\n[+] Scan Results for: ' + tgtName[0])
    except:
        print('\n[+] Scan Results for: ' + tgtIP)
    setdefaulttimeout(1)
    for tgtPort in tgtPorts:
        t = Thread(target=connScan, arg=(tgtHost, int(tgtPort)))
        t.start()
        print('Scanning Port ' + tgtPort)
        connScan(tgtHost, int(tgtPort))
def main():
    parser = optparse.OptionParser("usage%prog " +\
                                  "-H <targethost> -p <target port>")
    parser.add_option('-H', dest='tgtHost', type='string', help='specify target host')
    parser.add_option('-p', dest='tgtPort', type='string', help='specifiy target port[s] separated by comma')
    
    (options, args) = parser.parse_args()
    tgtHost = options.tgtHost
    tgtPorts = str(options.tgtPort).split(', ')
    if (tgtHost == None ) | (tgtPorts[0] == None):
        print('[-] Youe must specify a target host and port[s].')
        exit(0)
    portScan(tgtHost, tgtPorts)
    if __name__ == '__main__':
        main()
""""""
Try running this is the command line with flags for IP and ports
""""""

 Since connScan will print output the the screen we risk missing some output and put printing to the screen on lockdown until the data execution is done

Prior to printing the output `screenLock.Acquire()` is used. 
the *semaphore* will grant access to process and will print to the screen
if locked it will wait until threading is in control of the semaphore.
This will make sure only one thread is in control at a time 

In [ ]:
ScreenLock = Semaphore(value=1)
def connScan(tgtHost, tgtPort):
    try:
        connSkt = socket(AF_INET, SOCK_STREAM)
        connSkt.connect((tgtHost, tgtPort))
        connSkt.send('ViolentPython\r\n')
        results = connSkt.recv(100)
        screenLock.acquire()
        print '[+]%d/tcp open'% tgtPort
        print '[+] ' + str(results)
    except:
        screenLock.acquire()
        print '[-]%d/tcp closed'% tgtPort
    finally:
        screenLock.release()
        connSkt.close()

## Integrating NMap Port Scanner
----
